In [2]:
from matplotlib import pyplot as plt
import os
from PIL import Image
import numpy as np
import torch

def test(pipeline, save_path):
    # 指定使用 GPU 1
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    pipeline = pipeline.to(device)

    # 确保所有计算都在指定的设备上进行
    with torch.cuda.device(device):
        images = pipeline(batch_size=16,  # 假设我们想要保存16张图像
                          num_inference_steps=1000,
                          output_type='numpy').images

    # 将模型移回 CPU 并清空 GPU 缓存
    pipeline.to('cpu')
    torch.cuda.empty_cache()

    # 将图像数据转换为 uint8 类型，以便于保存
    images = (images * 255).round().astype('uint8')

    # 创建保存图像的目录，如果不存在的话
    os.makedirs(save_path, exist_ok=True)

    # 保存每个生成的图像
    for i, img in enumerate(images):
        # 将 NumPy 数组转换为 PIL Image 对象
        pil_image = Image.fromarray(img.astype('uint8'), 'RGB')
        # 定义每个图像的保存路径
        file_path = os.path.join(save_path, f"generated_image_{i}.png")
        # 保存图像
        pil_image.save(file_path)

    # 创建一个足够大的图像画布，以容纳4x4的图像
    canvas_width = images.shape[2] * 4
    canvas_height = images.shape[1] * 4
    new_im = Image.new('RGB', (canvas_width, canvas_height))

    # 将每张图像粘贴到大图像上
    for i in range(16):
        x = i % 4
        y = i // 4
        new_im.paste(Image.fromarray(images[i]), (x * images.shape[2], y * images.shape[1]))

    # 定义并保存拼接后的大图像的路径
    stiched_image_path = os.path.join(save_path, "stitched_image.png")
    new_im.save(stiched_image_path)

    # 如果你还想显示图像，可以取消下面这行的注释
    # plt.figure(figsize=(12, 9))
    # for i in range(16):  # 假设我们想要显示 16 张图像
    #     plt.subplot(4, 4, i + 1)
    #     plt.imshow(images[i], cmap='gray' if images[i].ndim == 2 else None)  # 使用灰度色图如果是灰度图
    #     plt.axis('off')
    # plt.tight_layout()
    # plt.show()

# 假设 pipeline 是一个已经定义好的模型或者处理流程
# test(your_pipeline, 'path_to_save_images')

In [1]:
from diffusers import DDPMPipeline, UNet2DModel, DDPMScheduler

#定义模型,随机初始化参数
model = UNet2DModel(
    sample_size=64,
    in_channels=3,
    out_channels=3,
    layers_per_block=3,#2
    block_out_channels=(128, 128, 256, 256, 512, 512),
    down_block_types=(
        'DownBlock2D',
        'DownBlock2D',
        'DownBlock2D',
        'DownBlock2D',
        'AttnDownBlock2D',
        'DownBlock2D',
    ),
    up_block_types=(
        'UpBlock2D',
        'AttnUpBlock2D',
        'UpBlock2D',
        'UpBlock2D',
        'UpBlock2D',
        'UpBlock2D',
    ),
)

#初始化工具类
scheduler = DDPMScheduler(num_train_timesteps=1000,
                          beta_schedule='linear',
                          prediction_type='epsilon')



In [3]:
# 预定义所有要使用的预训练模型的名称
pipeline_names = [
    'ddpm-bored-apes-128',
    'ddpm-flowers-128-100-hu-80-120',
    'ddpm-butterflies-128-150',
    'ddpm-pokemons-128-400',
    #'ddpm-glasses-128-100-120',
    #'ddpm-cats1-128-50-120-240',
    #'ddpm-arts-128-300',
    'ddpm-ema-cat-256',
    #'ddpm-church-finetune-wikiart-256',
    #'sd-class-metfaces-64-clean'
]

# 循环一百次
for i in range(100):
    # 创建一个新的文件夹路径，用于保存这一次所有模型生成的图像
    new_folder_path = f'aaaaasavetest/generated_images_{i}'
    # 确保路径存在，如果不存在则创建
    os.makedirs(new_folder_path, exist_ok=True)
    
    # 对于每个预训练模型，调用 test 函数
    for pipeline_name in pipeline_names:
        # 加载预训练模型
        pipeline = DDPMPipeline.from_pretrained(pipeline_name)
        # 定义每个模型生成图像的保存路径
        model_save_path = os.path.join(new_folder_path, f'{pipeline_name}_images')
        # 调用 test 函数，传入模型和保存路径
        test(pipeline, model_save_path)

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [14]:
#test(DDPMPipeline.from_pretrained('./ddpm-church-finetune-wikiart-256'),'wikiarts')

In [15]:
#test(DDPMPipeline.from_pretrained('./sd-class-metfaces-64-clean'))

In [16]:

#test(DDPMPipeline.from_pretrained('ddpm-ema-heightmap-512-10m'))

In [17]:
#test(DDPMPipeline.from_pretrained('ddpm-church-finetune-wikiart-256'))

In [18]:
#test(DDPMPipeline.from_pretrained('sd-class-metfaces-64-clean'),'face')

In [19]:
#test(DDPMPipeline.from_pretrained('ddpm-ema-flowers-64-test'),'flower')

In [20]:
#test(DDPMPipeline.from_pretrained('ddpm-ema-cat-256'),'cat')

In [21]:
test(DDPMPipeline.from_pretrained('ddpm-bored-apes-128'),'boredapes')

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [24]:
test(DDPMPipeline.from_pretrained('ddpm-bored-apes-128'),'boredapestest')

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [22]:
test(DDPMPipeline.from_pretrained('ddpm-flowers-128-100-hu-80-120'),'flowersfinal')

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [23]:
test(DDPMPipeline.from_pretrained('ddpm-butterflies-128-150'),'butterfliesfinal')

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [25]:
test(DDPMPipeline.from_pretrained('ddpm-bored-apes-128'),'boredapes1')
test(DDPMPipeline.from_pretrained('ddpm-flowers-128-100-hu-80-120'),'flowersfinal1')
test(DDPMPipeline.from_pretrained('ddpm-butterflies-128-150'),'butterfliesfinal1')

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
'''for i in range(100):
    # 创建一个新的文件夹路径，用于保存这一次test的结果
    new_folder_path = f'path_to_save_images/generated_images_{i}'
    # 确保路径存在，如果不存在则创建
    os.makedirs(new_folder_path, exist_ok=True)
    
    # 调用test函数，传入模型和新创建的文件夹路径
    test(DDPMPipeline.from_pretrained('ddpm-bored-apes-128'),'aaaboredapesepoch')
    test(DDPMPipeline.from_pretrained('ddpm-flowers-128-100-hu-80-120'),'aaaflowersfinalepoch')
    test(DDPMPipeline.from_pretrained('ddpm-butterflies-128-150'),'aaabutterfliesfinalepoch')
    test(DDPMPipeline.from_pretrained('ddpm-pokemons-128-400'),'aaapokemonfinalepoch')
    test(DDPMPipeline.from_pretrained('ddpm-glasses-128-100-120'),'aaaglassesfinalepoch')
    test(DDPMPipeline.from_pretrained('ddpm-cats1-128-50-120-240'),'aaacatsfinalepoch')
    test(DDPMPipeline.from_pretrained('ddpm-arts-128-300'),'aaaartsfinalepoch')
    test(DDPMPipeline.from_pretrained('ddpm-ema-cat-256'),'aaaemacatsfinalepoch')
    test(DDPMPipeline.from_pretrained('ddpm-church-finetune-wikiart-256'),'aaawikiartsfinalepoch')
    test(DDPMPipeline.from_pretrained('sd-class-metfaces-64-clean'),'aaafacesfinalepoch')'''
# 预定义所有要使用的预训练模型的名称
pipeline_names = [
    'ddpm-bored-apes-128',
    'ddpm-flowers-128-100-hu-80-120',
    'ddpm-butterflies-128-150',
    'ddpm-pokemons-128-400',
    'ddpm-glasses-128-100-120',
    'ddpm-cats1-128-50-120-240',
    'ddpm-arts-128-300',
    'ddpm-ema-cat-256',
    'ddpm-church-finetune-wikiart-256',
    'sd-class-metfaces-64-clean'
]

# 循环一百次
for i in range(100):
    # 创建一个新的文件夹路径，用于保存这一次所有模型生成的图像
    new_folder_path = f'aaaasavetest/generated_images_{i}'
    # 确保路径存在，如果不存在则创建
    os.makedirs(new_folder_path, exist_ok=True)
    
    # 对于每个预训练模型，调用 test 函数
    for pipeline_name in pipeline_names:
        # 加载预训练模型
        pipeline = DDPMPipeline.from_pretrained(pipeline_name)
        # 定义每个模型生成图像的保存路径
        model_save_path = os.path.join(new_folder_path, f'{pipeline_name}_images')
        # 调用 test 函数，传入模型和保存路径
        test(pipeline, model_save_path)

# 注意：确保 'path_to_save_images' 是你希望存放所有生成图像的父目录的实际路径。

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [28]:
#import os

# 假设 test 函数的定义如下：
# def test(pipeline, save_path):
#     # 这里是 test 函数的实现，负责使用 pipeline 生成图像并保存到 save_path

# 假设 DDPMPipeline.from_pretrained 是一个有效的函数调用，用于加载预训练模型

# 预定义所有要使用的预训练模型的名称
pipeline_names = [
    'ddpm-bored-apes-128',
    'ddpm-flowers-128-100-hu-80-120',
    'ddpm-butterflies-128-150',
    'ddpm-pokemons-128-400',
    'ddpm-glasses-128-100-120',
    'ddpm-cats1-128-50-120-240',
    'ddpm-arts-128-300',
    'ddpm-ema-cat-256',
    'ddpm-church-finetune-wikiart-256',
    'sd-class-metfaces-64-clean'
]

# 循环一百次
for i in range(100):
    # 创建一个新的文件夹路径，用于保存这一次所有模型生成的图像
    new_folder_path = f'savetest/generated_images_{i}'
    # 确保路径存在，如果不存在则创建
    os.makedirs(new_folder_path, exist_ok=True)
    
    # 对于每个预训练模型，调用 test 函数
    for pipeline_name in pipeline_names:
        # 加载预训练模型
        pipeline = DDPMPipeline.from_pretrained(pipeline_name)
        # 定义每个模型生成图像的保存路径
        model_save_path = os.path.join(new_folder_path, f'{pipeline_name}_images')
        # 调用 test 函数，传入模型和保存路径
        test(pipeline, model_save_path)

# 注意：确保 'path_to_save_images' 是你希望存放所有生成图像的父目录的实际路径。

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
test(DDPMPipeline.from_pretrained('ddpm-bored-apes-128'),'boredapes2')
test(DDPMPipeline.from_pretrained('ddpm-flowers-128-100-hu-80-120'),'flowersfinal2')
test(DDPMPipeline.from_pretrained('ddpm-butterflies-128-150'),'butterfliesfinal2')

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
test(DDPMPipeline.from_pretrained('ddpm-bored-apes-128'),'boredapes3')
test(DDPMPipeline.from_pretrained('ddpm-flowers-128-100-hu-80-120'),'flowersfinal3')
test(DDPMPipeline.from_pretrained('ddpm-butterflies-128-150'),'butterfliesfinal3')

In [ ]:
# 预定义所有要使用的预训练模型的名称
pipeline_names = [
    'ddpm-bored-apes-128',
    'ddpm-flowers-128-100-hu-80-120',
    'ddpm-butterflies-128-150',
    'ddpm-pokemons-128-400',
    'ddpm-glasses-128-100-120',
    'ddpm-cats1-128-50-120-240',
    'ddpm-arts-128-300',
    'ddpm-ema-cat-256',
    'ddpm-church-finetune-wikiart-256',
    'sd-class-metfaces-64-clean'
]

# 循环一百次
for i in range(100):
    # 创建一个新的文件夹路径，用于保存这一次所有模型生成的图像
    new_folder_path = f'aaaaasavetest/generated_images_{i}'
    # 确保路径存在，如果不存在则创建
    os.makedirs(new_folder_path, exist_ok=True)
    
    # 对于每个预训练模型，调用 test 函数
    for pipeline_name in pipeline_names:
        # 加载预训练模型
        pipeline = DDPMPipeline.from_pretrained(pipeline_name)
        # 定义每个模型生成图像的保存路径
        model_save_path = os.path.join(new_folder_path, f'{pipeline_name}_images')
        # 调用 test 函数，传入模型和保存路径
        test(pipeline, model_save_path)

# 注意：确保 'path_to_save_images' 是你希望存放所有生成图像的父目录的实际路径。